In [54]:
import pandas as pd 
import numpy as np 
import glob
from Utils_ETL import helper as hpe

Para una muestra representativa, voy a tomar Texas ya que es donde están los locales más importantes de Texas Roadhouse, además de ser uno de los lugares dónde es más tradicional esta comida

In [4]:
df_reviews = hpe.merge_dataframes_ETL("datasets/review-Texas")

In [5]:
df_meta1 = hpe.merge_dataframes_ETL("datasets/meta1")

In [7]:
df_meta2 = hpe.merge_dataframes_ETL("datasets/meta2")

In [8]:
df_meta3 = hpe.merge_dataframes_ETL("datasets/meta3")

In [9]:
df_meta4 = hpe.merge_dataframes_ETL("datasets/meta4")

In [27]:
df_meta1 = df_meta1.explode("category")
df_meta2 = df_meta2.explode("category")
df_meta3 = df_meta3.explode("category")
df_meta4 = df_meta4.explode("category")

Vamos a ver que categorias tienen los restaurantes Texas Roadhouse y las anotamos

In [ ]:
#guia para ver en que se basó la funcion
lista_categoria = []
mask = df_meta2["name"] == "Texas Roadhouse"
TexRod = df_meta2[mask]
categorias = TexRod.category.values
for categoria in categorias:
    if categoria in lista_categoria:
        pass
    else:
        lista_categoria.append(categoria)
lista_categoria

In [53]:
valores1 = hpe.associated_values_other_columns(df_meta1,"name","Texas Roadhouse","category")
valores2 = hpe.associated_values_other_columns(df_meta2,"name","Texas Roadhouse","category")
valores3 = hpe.associated_values_other_columns(df_meta3,"name","Texas Roadhouse","category")
valores4 = hpe.associated_values_other_columns(df_meta4,"name","Texas Roadhouse","category")
valores = valores1 + valores2 + valores3 + valores4
valores = list(set(valores))
valores

['Bar',
 'Steak house',
 'Restaurant',
 'New American restaurant',
 'American restaurant']

Vamos a filtrar solamente a los locales que tengan esas categorias para, despues, quedarnos solamente con las reviews de lugares parecidos a Texas Roadhouse, por categoria

In [ ]:
#guia para ver en que se basó la funcion
mask = df_meta4["category"].isin(valores)
meta_filtrado4 = df_meta4[mask]

In [61]:
meta_filtrado1 = hpe.filer_by_values_in_column(df_meta1,"category",valores)
meta_filtrado2 = hpe.filer_by_values_in_column(df_meta2,"category",valores)
meta_filtrado3 = hpe.filer_by_values_in_column(df_meta3,"category",valores)
meta_filtrado4 = hpe.filer_by_values_in_column(df_meta4,"category",valores)

In [62]:
meta_filtrado = pd.concat([meta_filtrado1, meta_filtrado2, meta_filtrado3,meta_filtrado4], ignore_index=True)

Estoy haciendo el ETL de reviews, no de los metadatos, por lo tanto me voy a quedar con las columnas que me sirver para ese proposito

In [74]:
metadata = meta_filtrado[["name","address","gmap_id","description","category","avg_rating","price"]]

,name,address,gmap_id,description,category,avg_rating,price
0,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,Restaurant,4.5,None
1,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",Restaurant,4.0,None
2,The Bar Stool,"The Bar Stool, 618 Main St, Highland, WI 53543",0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,None,Bar,4.5,None
3,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,Restaurant,4.4,None
4,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,None,Restaurant,5.0,$$
...,...,...,...,...,...,...,...
148720,Country Kitchen,"Country Kitchen, 701 Main St, Pangburn, AR 72121",0x87d24cd30b7695d1:0xdc25418ff311a6aa,Homey chain eatery serving breakfast all day &...,American restaurant,4.5,$
148721,"Simple Simon's Pizza - Vilonia, AR","Simple Simon's Pizza - Vilonia, AR, 14 S Churc...",0x87d28d5aa640a9c5:0x3c9cc5d3a99de80c,None,Restaurant,4.5,None
148722,Subpreme Grill,"Subpreme Grill, 15260 SW 280th St UNIT 111, Ho...",0x88d9dd6e78851851:0x849dcf273582261c,None,Restaurant,4.3,$
148723,Reuben's Marc,"Reuben's Marc, 8131 Stenton Ave, Philadelphia,...",0x89c6b993431d9a19:0xea378de3c317996e,None,Bar,4.0,$


Unimos los dataframes

In [75]:
df_reviews_meta = pd.merge(df_reviews, metadata, on='gmap_id', how='left')

In [76]:
df_reviews_meta

,user_id,name_x,time,rating,text,pics,resp,gmap_id,name_y,address,description,category,avg_rating,price
0,1.105453e+20,Kimberly Feger,1625431734826,5,"The pharmacist, Erin, is phenomenal. She was s...",None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
1,1.036198e+20,Briana Streit,1568336283913,2,I gave them 2 stars because they offer prescri...,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
2,1.019167e+20,Sylvia Caudillo,1519873586872,1,If I could put minus stars I would. This has t...,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
3,1.178709e+20,Ginger Kinyon,1571638384637,1,Please fix your restroom doors,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
4,1.103973e+20,Angeles Arellano,1546568877087,1,This pharmacy Walmart dose not work not come a...,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430180,1.021681e+20,t williams,1562883151018,5,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN
2430181,1.054173e+20,Erika Lira,1532009672494,5,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN
2430182,1.107857e+20,Jon Brent,1567968578697,5,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN
2430183,1.091676e+20,lunna cabal,1506476840285,4,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
#cambiamos los nombres confusos
df_reviews_meta = df_reviews_meta.rename(columns={'name_x': 'name',
                                                  "name_y": "local"})

Nos quedamos solo con los lugares con la misma categoria de nuestro cliente

In [91]:
df_reviews_meta_filtered =  hpe.null_filter_by_columns(df_reviews_meta,"category")

In [92]:
df_reviews_meta_filtered

,user_id,name,time,rating,text,pics,resp,gmap_id,local,address,description,category,avg_rating,price
57,1.089906e+20,Karina Asher,1630618611013,5,If you’ve seen Harold and Kumar Go To White Ca...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
58,1.089906e+20,Karina Asher,1630618611013,5,If you’ve seen Harold and Kumar Go To White Ca...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
59,1.166907e+20,Robert Stevens,1630859678572,5,Had the sliders and onion rings for the first ...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1630860556690, 'text': 'Thanks for yo...",0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
60,1.166907e+20,Robert Stevens,1630859678572,5,Had the sliders and onion rings for the first ...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1630860556690, 'text': 'Thanks for yo...",0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
61,1.113003e+20,Kyle Johnson,1630623576851,3,I appreciate Golden Castle trying something ne...,None,None,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429635,1.156814e+20,Zoraida Vences,1611626536005,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
2429636,1.153003e+20,Gabriel Gonzales,1607396954553,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
2429637,1.009257e+20,Jacqueline Cano,1569606978949,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
2429638,1.091067e+20,Manuel Martinez,1607657246694,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
